In [3]:
import pandas as pd
import pyodbc
import sqlite3

In [56]:
DB = {"servername": "localhost\SQLEXPRESS",
      "database": "greatoutdoors"}

export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + 
                             ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')

export_cursor = export_conn.cursor()


# Load utilities


# Load tables

sales_db = sqlite3.connect("Databases/GreatOutdoors/go_sales.sqlite")
product = pd.read_sql_query("SELECT * FROM product;", sales_db)
country = pd.read_sql_query("SELECT * FROM country;", sales_db)
product_line = pd.read_sql_query("SELECT * FROM product_line;", sales_db)

sales_branch = pd.read_sql_query("SELECT * FROM sales_branch;", sales_db)
retailer_site = pd.read_sql_query("SELECT * FROM retailer_site;", sales_db)
product_type = pd.read_sql_query("SELECT * FROM product_type;", sales_db)
returns = pd.read_sql_query("SELECT * FROM returned_item;", sales_db)
order_details = pd.read_sql_query("SELECT * FROM order_details;", sales_db)
sales_staff = pd.read_sql_query("SELECT * FROM sales_staff;", sales_db)
order_header = pd.read_sql_query("SELECT * FROM order_header;", sales_db)
returned_item  = pd.read_sql_query("SELECT * FROM returned_item;", sales_db)
return_reason  = pd.read_sql_query("SELECT * FROM return_reason;", sales_db)
target = pd.read_sql_query("SELECT * FROM SALES_TARGETData;", sales_db)
order_method = pd.read_sql_query("SELECT * FROM order_method;", sales_db)



for row in export_cursor.tables():
    print(row.table_name)


<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
C:\Users\robbi\AppData\Local\Temp\ipykernel_1208\716185545.py:1: SyntaxWarning: invalid escape sequence '\S'
  DB = {"servername": "localhost\SQLEXPRESS",


course
date
DAY
MONTH
order_method
orders
Product
Retailer
retailer_site
sales_staff
SALES_TargetData
SALESDATA
satisfaction
satisfaction_type
training
YEAR
trace_xe_action_map
trace_xe_event_map
CHECK_CONSTRAINTS
COLUMN_DOMAIN_USAGE
COLUMN_PRIVILEGES
COLUMNS
CONSTRAINT_COLUMN_USAGE
CONSTRAINT_TABLE_USAGE
DOMAIN_CONSTRAINTS
DOMAINS
KEY_COLUMN_USAGE
PARAMETERS
REFERENTIAL_CONSTRAINTS
ROUTINE_COLUMNS
ROUTINES
SCHEMATA
SEQUENCES
TABLE_CONSTRAINTS
TABLE_PRIVILEGES
TABLES
VIEW_COLUMN_USAGE
VIEW_TABLE_USAGE
VIEWS
all_columns
all_objects
all_parameters
all_sql_modules
all_views
allocation_units
assemblies
assembly_files
assembly_modules
assembly_references
assembly_types
asymmetric_keys
availability_databases_cluster
availability_group_listener_ip_addresses
availability_group_listeners
availability_groups
availability_groups_cluster
availability_read_only_routing_lists
availability_replicas
backup_devices
certificates
change_tracking_databases
change_tracking_tables
check_constraints
column_e

In [ ]:
# Product


transformed_product = []

transformed_product = product.merge(product_type, 'inner', 'PRODUCT_TYPE_CODE').merge(product_line, 'inner', 'PRODUCT_LINE_CODE').rename(columns={
    "INTRODUCTION_DATE": "INTRODUCTION_DATE_date",
    "PRODUCT_TYPE_CODE": "PRODUCT_TYPE_CODE",
    "PRODUCTION_COST": "PRODUCTION_COST_number",
    "MARGIN": "MARGIN_NUMBER",
    "PRODUCT_IMAGE": "PRODUCT_IMAGE_image",
    "LANGUAGE": "LANGUAGE_language",
    "PRODUCT_TYPE_EN": "PRODUCT_TYPE_name",
    "DESCRIPTION": "DESCRIPTION_description",
}).drop(columns="TRIAL888_x", axis=1).drop(columns="TRIAL888_y", axis=1).drop(columns="TRIAL888", axis=1)


for index, row in transformed_product.iterrows():
    try:
        query = f"INSERT INTO Product VALUES ({row['PRODUCT_NUMBER']},  '{row['PRODUCT_NAME']}', '{ convert_date_format(row['INTRODUCTION_DATE_date']) }', {row['PRODUCT_TYPE_CODE']}, '{row['PRODUCT_TYPE_name']}', {row['PRODUCT_LINE_CODE']}, '{row['PRODUCT_LINE_EN']}', {row['PRODUCTION_COST_number']}, {row['MARGIN_NUMBER']},'{row['PRODUCT_IMAGE_image']}', '{row['LANGUAGE_language']}', '{escape_single_quotes(row['DESCRIPTION_description'])}');"
        export_cursor.execute(query)
        print(query)
    except pyodbc.Error as e:
        print(query)
        # print(e)

    export_cursor.commit()


In [ ]:
# Order


merged_df = pd.merge(order_header, order_details, on='ORDER_NUMBER', how='inner')
order = pd.merge(merged_df, returned_item, on='ORDER_DETAIL_CODE', how = 'inner')

for index, row in order.iterrows():
    try:
        query = f"INSERT INTO 'orders' VALUES ({row['ORDER_NUMBER']},  '{escape_single_quotes(row['RETAILER_NAME'])}', '{row['ORDER_DATE']}', 1, 1, 2024, {row['ORDER_DETAIL_CODE']}, {row['PRODUCT_NUMBER']}, {row['QUANTITY']}, {row['UNIT_COST']}, {row['UNIT_PRICE']}, {row['UNIT_SALE_PRICE']}, {row['RETURN_CODE']}, '{convert_date_format(strip_time_from_string(row['RETURN_DATE']))}', {row['RETURN_REASON_CODE']}, {row['RETURN_QUANTITY']}, {row['ORDER_METHOD_CODE']}, {row['SALES_STAFF_CODE']}, {row['SALES_BRANCH_CODE']}, {row['RETAILER_SITE_CODE']});"
        print(query)
    except pyodbc.Error as e:
        print(query)
        print(e)
        
order

In [ ]:
# Sales Forecast & Inventory

sales_data_forecast = pd.read_csv("Databases/GreatOutdoors/GO_SALES_PRODUCT_FORECASTData.csv")
sales_inventory_levels = pd.read_csv("Databases/GreatOutdoors/GO_SALES_INVENTORY_LEVELSData.csv")

sales_data_merged = pd.merge(sales_data_forecast, sales_inventory_levels, 
                     left_on=['PRODUCT_NUMBER','YEAR','MONTH'],
                     right_on=['PRODUCT_NUMBER','INVENTORY_YEAR','INVENTORY_MONTH'],
                     how='inner')



sales_data_merged = sales_data_merged.rename(columns={'YEAR': 'FORECAST_YEAR', 'MONTH': 'FORECAST_MONTH'})



for index, row in sales_data_merged.iterrows():
    try:
        query = f"INSERT INTO 'SALESDATA' VALUES ({row['PRODUCT_NUMBER']}, {row['FORECAST_MONTH']}, {row['FORECAST_YEAR']}, {row['INVENTORY_MONTH']}, {row['INVENTORY_YEAR']}, {row['INVENTORY_COUNT']});"
        print(query)
    except pyodbc.Error as e:
        print(query)
        print(e)


# len(sales_data_merged)




In [ ]:
# Order Method

for index, row in order_method.iterrows():
    try:
        query = f"INSERT INTO order_method VALUES ({row['ORDER_METHOD_CODE']}, '{row['ORDER_METHOD_EN']}');"
        print(query)
    except pyodbc.Error as e:
        print(query)
        print(e)

In [ ]:
#Retailer
crm_db = sqlite3.connect("Databases/GreatOutdoors/go_crm.sqlite")

retailer = pd.read_sql_query("SELECT * FROM retailer;", crm_db)
retailer_type = pd.read_sql_query("Select * FROM retailer_type", crm_db)
retailer_segment = pd.read_sql_query("select * from retailer_segment", crm_db)
retailer_headquarters = pd.read_sql_query("select * from retailer_headquarters", crm_db)

merged_df = pd.merge(retailer,retailer_type,on="RETAILER_TYPE_CODE",how="left")
merged_df2 = pd.merge(merged_df,retailer_type,on="RETAILER_TYPE_CODE", how="left")
merged_df3 = pd.merge(merged_df2,retailer_headquarters,on="RETAILER_CODEMR", how = "left")
retailer_segment = retailer_segment.rename(columns={'TRIAL222': 'TRIAL222_segment'})
merged_df4 = pd.merge(merged_df3,retailer_segment,on="SEGMENT_CODE",how="left")

merged_df4

In [53]:
sql_column_names = ['RETAILER_CODE', 'RETAILER_company_name', 'RETAILER_TYPE_CODE', 'RETAILER_TYPE_name','SEGMENT_CODE','SEGMENT_NAME','SEGMENT_DESCRIPTION','RETAILER_NAME','ADDRESS1','ADDRESS2', 'CITY','REGION','COUNTRY_CODE','RETAILER_CODEMR' ]
merged_df4_reordered = merged_df4[sql_column_names]
merged_df4_reordered

,RETAILER_CODE,RETAILER_company_name,RETAILER_TYPE_CODE,RETAILER_TYPE_name,SEGMENT_CODE,SEGMENT_NAME,SEGMENT_DESCRIPTION,RETAILER_NAME,ADDRESS1,ADDRESS2,CITY,REGION,COUNTRY_CODE,RETAILER_CODEMR
0,1,ActiForme,5,Equipment Rental Store,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1,100,Golf Masters,1,Golf Shop,3,Golf Shop,Primarily offers golf equipment and accessories.,Golf Masters,2845 South Second Street,None,Lincoln,Nebraska,3,70
2,101,The Marketplace,2,Department Store,5,Department Store,Offers a wide range of goods.,The Marketplace,1902 Lancaster Street,None,New York,New York,3,71
3,102,Camping Equipment Online,3,Direct Marketing,7,Direct Marketing,Offers a wide range of goods through mail orde...,Camping Equipment Online,332 South Main Street,P.O. Box 224,Hanover,New Hampshire,3,72
4,103,Tamarack Outfitter Rentals,5,Equipment Rental Store,9,Equipment Rental Store,Offers various sports and outdoors equipment f...,Tamarack Outfitter Rentals,582 NE Tomahawk Island Drive,None,Portland,Oregon,3,73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,95,Falcon Outfitters,6,Outdoors Shop,1,Outdoors Shop - Exclusive,"Primarily offers camping supplies, hiking equi...",Falcon Outfitters,9755 Riverside Drive,None,Ottawa,Ontario,4,5
105,96,Maximum Sports,8,Sports Store,12,Sports Store,Primarily offers a wide range of sporting goods.,Maximum Sports,"Suite 12, 801 Dundas Street",None,Toronto,Ontario,4,6
106,97,Act'N'Up Fitness,4,Warehouse Store,8,Warehouse Store,Offers a wide range of goods in a large wareho...,Act'N'Up Fitness,"87, 2399 Sommerset Avenue",None,Toronto,Ontario,4,7
107,98,Advanced Climbing Ltd,5,Equipment Rental Store,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [64]:
DB = {"servername": "localhost\\SQLEXPRESS", "database": "greatoutdoors"}

export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
export_cursor = export_conn.cursor()

for index, row in merged_df4_reordered.iterrows():
    try:
        row = row.fillna('NULL')
        row = row.astype(str).apply(lambda x: x.replace("'", "''"))
        query = f"""
            INSERT INTO Retailer 
            VALUES (
                {row['RETAILER_CODE']}, 
                '{row['RETAILER_company_name']}', 
                '{row['RETAILER_TYPE_CODE']}', 
                '{row['RETAILER_TYPE_name']}', 
                '{row['SEGMENT_CODE']}', 
                '{row['SEGMENT_NAME']}', 
                '{row['SEGMENT_DESCRIPTION']}', 
                '{row['RETAILER_NAME']}', 
                '{row['ADDRESS1']}', 
                '{row['ADDRESS2']}', 
                '{row['CITY']}', 
                '{row['REGION']}', 
                '{row['COUNTRY_CODE']}', 
                {row['RETAILER_CODEMR']}
            )
        """
        export_cursor.execute(query)  # Execute the query
        export_conn.commit()  # Commit the transaction to save changes
        print("Row inserted successfully.")
    except pyodbc.Error as e:
        print("Error occurred while inserting row:")
        print(query)
        print(e)

# Close the cursor and connection when done
export_cursor.close()
export_conn.close()


Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
R